In [1]:
import numpy as np 
import pandas as pd 
import os
import json
import cv2
import matplotlib.pyplot as plt
from skimage import io
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model 
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard
import keras
import matplotlib.pyplot as plt


IMAGES_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Images"
df = pd.read_csv("/kaggle/input/glaucoma-datasets/G1020/G1020.csv")
HEIGHT, WIDTH = 300, 300
NUM_CLASSES = 2
BATCH_SIZE = 16
EPOCHS = 150
LR = 1e-4
print(df)

             imageID  binaryLabels
0        image_0.jpg             0
1        image_1.jpg             0
2        image_3.jpg             0
3        image_4.jpg             0
4        image_5.jpg             0
...              ...           ...
1015  image_3198.jpg             0
1016  image_3199.jpg             0
1017  image_3201.jpg             1
1018  image_3202.jpg             1
1019  image_2568.jpg             0

[1020 rows x 2 columns]


In [2]:
def load_data(df, image_size=(HEIGHT, WIDTH)):
    images, labels = [], []
    print(f"Total images in dataframe: {len(df)}")

    for index, row in df.iterrows():
        img_name = row['imageID']
        label = row['binaryLabels']
        img_path = os.path.join(IMAGES_FOLDER, img_name)

        if os.path.exists(img_path):
            image = cv2.imread(img_path, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, image_size)
            images.append(image)
            labels.append(label)
    
    return np.array(images), np.array(labels)

images, labels = load_data(df)
images = np.array(images) / 255.0
labels = to_categorical(labels, num_classes=NUM_CLASSES)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, stratify=labels, random_state=42)

print(f"Dataset Shape: {images.shape}")
print(f"Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=90,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   brightness_range=[0.8, 1.2],
                                   shear_range=0.2)


test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_generator = test_datagen.flow(
    X_test,
    y_test,
    batch_size=BATCH_SIZE,
    shuffle=False
)


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))

for layer in base_model.layers[:-10]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)  
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=LR), loss='categorical_crossentropy', metrics=["accuracy"])


checkpoint = ModelCheckpoint("resnet50_model.keras", monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tensorboard = TensorBoard(log_dir="./logs")
callbacks_list = [checkpoint, tensorboard]

model.fit(train_generator, epochs=EPOCHS, validation_data=test_generator, callbacks=[checkpoint, reduce_lr, early_stop])

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

Total images in dataframe: 1020
Dataset Shape: (1020, 300, 300, 3)
Train Shape: (714, 300, 300, 3), Test Shape: (306, 300, 300, 3)
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/150


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.4616 - loss: 0.7710
Epoch 1: val_accuracy improved from -inf to 0.29085, saving model to resnet50_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 44s 575ms/step - accuracy: 0.4661 - loss: 0.7685 - val_accuracy: 0.2908 - val_loss: 0.9709 - learning_rate: 1.0000e-04
Epoch 2/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step - accuracy: 0.6946 - loss: 0.6770
Epoch 2: val_accuracy did not improve from 0.29085
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 295ms/step - accuracy: 0.6944 - loss: 0.6770 - val_accuracy: 0.2908 - val_loss: 1.0353 - learning_rate: 1.0000e-04
Epoch 3/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - accuracy: 0.6917 - loss: 0.6384
Epoch 3: val_accuracy did not improve from 0.29085
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 298ms/step - accuracy: 0.6915 - loss: 0.6386 - val_accuracy: 0.2908 - val_loss: 0.9496 - learning_rate: 1.0000e-04
Epoch 4/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - accuracy: 0.7133 - loss: 0.6339
Epoch 4: val_accuracy did not impr